In [1]:
import pandas as pd
import glob
import os
import numpy as np

In [2]:
import datetime
import timestring
from dateutil.relativedelta import relativedelta

import plotly.express as pt

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (12,6)

import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_hdf('input_t_mat.h5')

In [4]:
df

,ID,checkout,checkout_at,customer_no,day,enter_at,entry,location,timestamp,week_code,time_spent,time_spent_m,sequence
0,000010001,False,2019-09-04 07:02:00,1,wednesday,2019-09-04 07:00:00,True,checkin,2019-09-04 06:59:59,00001,0 days 00:02:00,2,0
1,000010001,False,2019-09-04 07:02:00,1,wednesday,2019-09-04 07:00:00,True,fruit,2019-09-04 07:00:00,00001,0 days 00:02:00,2,1
2,000010001,True,2019-09-04 07:02:00,1,wednesday,2019-09-04 07:00:00,False,checkout,2019-09-04 07:02:00,00001,0 days 00:02:00,2,2
3,000010002,False,2019-09-04 07:06:00,2,wednesday,2019-09-04 07:00:00,True,checkin,2019-09-04 06:59:59,00001,0 days 00:06:00,6,0
4,000010002,False,2019-09-04 07:06:00,2,wednesday,2019-09-04 07:00:00,True,dairy,2019-09-04 07:00:00,00001,0 days 00:06:00,6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32319,100001509,True,2019-09-06 21:50:00,1509,friday,2019-09-06 21:50:00,True,drinks,2019-09-06 21:50:00,10000,0 days 00:00:00,0,1
32348,100001509,True,2019-09-06 21:50:00,1509,friday,2019-09-06 21:50:00,True,checkout,2019-09-06 21:51:00,10000,0 days 00:00:00,0,2
32320,100001510,False,2019-09-06 21:50:00,1510,friday,2019-09-06 21:50:00,True,checkin,2019-09-06 21:49:59,10000,0 days 00:00:00,0,0
32321,100001510,True,2019-09-06 21:50:00,1510,friday,2019-09-06 21:50:00,True,spices,2019-09-06 21:50:00,10000,0 days 00:00:00,0,1


In [5]:
df.groupby('day')['timestamp'].min()

day
friday      2019-09-06 06:59:59
monday      2019-09-02 07:02:59
thursday    2019-09-05 06:59:59
tuesday     2019-09-03 07:01:59
wednesday   2019-09-04 06:59:59
Name: timestamp, dtype: datetime64[ns]

In [6]:
df.groupby('day')['timestamp'].max()

day
friday      2019-09-06 21:51:00
monday      2019-09-02 21:51:00
thursday    2019-09-05 21:49:00
tuesday     2019-09-03 21:51:00
wednesday   2019-09-04 21:50:00
Name: timestamp, dtype: datetime64[ns]

In [7]:
df['timestamp'].loc[df['location'] == 'checkin'] = df['enter_at']

In [8]:
df['tod'] = pd.cut(pd.to_datetime(df['timestamp']).dt.hour,
       bins=[0, 6, 12, 18, 24],
       labels=['night', 'morning', 'afternoon', 'evening'],
       right=False,
       include_lowest=True)

In [9]:
df['tod'].unique()

['morning', 'afternoon', 'evening']
Categories (4, object): ['night' < 'morning' < 'afternoon' < 'evening']

In [10]:
df

,ID,checkout,checkout_at,customer_no,day,enter_at,entry,location,timestamp,week_code,time_spent,time_spent_m,sequence,tod
0,000010001,False,2019-09-04 07:02:00,1,wednesday,2019-09-04 07:00:00,True,checkin,2019-09-04 07:00:00,00001,0 days 00:02:00,2,0,morning
1,000010001,False,2019-09-04 07:02:00,1,wednesday,2019-09-04 07:00:00,True,fruit,2019-09-04 07:00:00,00001,0 days 00:02:00,2,1,morning
2,000010001,True,2019-09-04 07:02:00,1,wednesday,2019-09-04 07:00:00,False,checkout,2019-09-04 07:02:00,00001,0 days 00:02:00,2,2,morning
3,000010002,False,2019-09-04 07:06:00,2,wednesday,2019-09-04 07:00:00,True,checkin,2019-09-04 07:00:00,00001,0 days 00:06:00,6,0,morning
4,000010002,False,2019-09-04 07:06:00,2,wednesday,2019-09-04 07:00:00,True,dairy,2019-09-04 07:00:00,00001,0 days 00:06:00,6,1,morning
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32319,100001509,True,2019-09-06 21:50:00,1509,friday,2019-09-06 21:50:00,True,drinks,2019-09-06 21:50:00,10000,0 days 00:00:00,0,1,evening
32348,100001509,True,2019-09-06 21:50:00,1509,friday,2019-09-06 21:50:00,True,checkout,2019-09-06 21:51:00,10000,0 days 00:00:00,0,2,evening
32320,100001510,False,2019-09-06 21:50:00,1510,friday,2019-09-06 21:50:00,True,checkin,2019-09-06 21:50:00,10000,0 days 00:00:00,0,0,evening
32321,100001510,True,2019-09-06 21:50:00,1510,friday,2019-09-06 21:50:00,True,spices,2019-09-06 21:50:00,10000,0 days 00:00:00,0,1,evening


### T matrix for morning

In [11]:
df_morning = df[['timestamp', 'ID', 'location','sequence']].loc[df['tod'] == 'morning']

In [12]:
df_morning.sort_values(by=['ID', 'timestamp'])

,timestamp,ID,location,sequence
0,2019-09-04 07:00:00,000010001,checkin,0
1,2019-09-04 07:00:00,000010001,fruit,1
2,2019-09-04 07:02:00,000010001,checkout,2
3,2019-09-04 07:00:00,000010002,checkin,0
4,2019-09-04 07:00:00,000010002,dairy,1
...,...,...,...,...
27771,2019-09-06 11:58:00,100000470,checkin,0
27772,2019-09-06 11:58:00,100000470,fruit,1
27773,2019-09-06 11:59:00,100000470,checkout,2
27774,2019-09-06 11:58:00,100000471,checkin,0


In [13]:
df_morning['shift'] = df_morning.groupby(['ID'])['location'].shift(-1)

In [14]:
df_morning.sort_values(by=['ID', 'timestamp'])

,timestamp,ID,location,sequence,shift
0,2019-09-04 07:00:00,000010001,checkin,0,fruit
1,2019-09-04 07:00:00,000010001,fruit,1,checkout
2,2019-09-04 07:02:00,000010001,checkout,2,NaN
3,2019-09-04 07:00:00,000010002,checkin,0,dairy
4,2019-09-04 07:00:00,000010002,dairy,1,spices
...,...,...,...,...,...
27771,2019-09-06 11:58:00,100000470,checkin,0,fruit
27772,2019-09-06 11:58:00,100000470,fruit,1,checkout
27773,2019-09-06 11:59:00,100000470,checkout,2,NaN
27774,2019-09-06 11:58:00,100000471,checkin,0,dairy


In [15]:
round(pd.crosstab(df_morning['location'], df_morning['shift'], normalize=0), 2)

shift,checkout,dairy,drinks,fruit,spices
location,,,,,
checkin,0.00,0.30,0.15,0.37,0.18
dairy,0.40,0.00,0.21,0.19,0.19
drinks,0.53,0.02,0.00,0.22,0.22
fruit,0.50,0.24,0.13,0.00,0.12
spices,0.25,0.34,0.27,0.14,0.00


### T matrix for afternoon

In [16]:
df_afternoon = df[['timestamp', 'ID', 'location','sequence']].loc[df['tod'] == 'afternoon']

In [17]:
df_afternoon.sort_values(by=['ID', 'timestamp'])

,timestamp,ID,location,sequence
2178,2019-09-04 12:03:00,000010502,fruit,9
2179,2019-09-04 12:04:00,000010502,drinks,10
2180,2019-09-04 12:06:00,000010502,checkout,11
2202,2019-09-04 12:00:00,000010507,spices,7
2203,2019-09-04 12:02:00,000010507,drinks,8
...,...,...,...,...
30400,2019-09-06 17:59:00,100001076,dairy,1
30402,2019-09-06 17:59:00,100001077,checkin,0
30403,2019-09-06 17:59:00,100001077,drinks,1
30405,2019-09-06 17:59:00,100001078,checkin,0


In [18]:
df_afternoon['shift'] = df_afternoon.groupby(['ID'])['location'].shift(-1)

In [19]:
df_afternoon.sort_values(by=['ID', 'timestamp'])

,timestamp,ID,location,sequence,shift
2178,2019-09-04 12:03:00,000010502,fruit,9,drinks
2179,2019-09-04 12:04:00,000010502,drinks,10,checkout
2180,2019-09-04 12:06:00,000010502,checkout,11,NaN
2202,2019-09-04 12:00:00,000010507,spices,7,drinks
2203,2019-09-04 12:02:00,000010507,drinks,8,fruit
...,...,...,...,...,...
30400,2019-09-06 17:59:00,100001076,dairy,1,NaN
30402,2019-09-06 17:59:00,100001077,checkin,0,drinks
30403,2019-09-06 17:59:00,100001077,drinks,1,NaN
30405,2019-09-06 17:59:00,100001078,checkin,0,dairy


In [20]:
round(pd.crosstab(df_afternoon['location'], df_afternoon['shift'], normalize=0), 2)

shift,checkout,dairy,drinks,fruit,spices
location,,,,,
checkin,0.00,0.29,0.13,0.40,0.18
dairy,0.41,0.00,0.20,0.20,0.19
drinks,0.53,0.02,0.00,0.22,0.22
fruit,0.51,0.23,0.14,0.00,0.12
spices,0.24,0.32,0.27,0.16,0.00


### T matrix for evening

In [21]:
df_evening = df[['timestamp', 'ID', 'location','sequence']].loc[df['tod'] == 'evening']

In [22]:
df_evening.sort_values(by=['ID', 'timestamp'])

,timestamp,ID,location,sequence
4682,2019-09-04 18:01:00,000011093,fruit,6
4683,2019-09-04 18:02:00,000011093,drinks,7
4684,2019-09-04 18:04:00,000011093,spices,8
4685,2019-09-04 18:10:00,000011093,drinks,9
4686,2019-09-04 18:11:00,000011093,checkout,10
...,...,...,...,...
32319,2019-09-06 21:50:00,100001509,drinks,1
32348,2019-09-06 21:51:00,100001509,checkout,2
32320,2019-09-06 21:50:00,100001510,checkin,0
32321,2019-09-06 21:50:00,100001510,spices,1


In [23]:
df_evening['shift'] = df_evening.groupby(['ID'])['location'].shift(-1)

In [24]:
df_evening.sort_values(by=['ID', 'timestamp'])

,timestamp,ID,location,sequence,shift
4682,2019-09-04 18:01:00,000011093,fruit,6,drinks
4683,2019-09-04 18:02:00,000011093,drinks,7,spices
4684,2019-09-04 18:04:00,000011093,spices,8,drinks
4685,2019-09-04 18:10:00,000011093,drinks,9,checkout
4686,2019-09-04 18:11:00,000011093,checkout,10,NaN
...,...,...,...,...,...
32319,2019-09-06 21:50:00,100001509,drinks,1,checkout
32348,2019-09-06 21:51:00,100001509,checkout,2,NaN
32320,2019-09-06 21:50:00,100001510,checkin,0,spices
32321,2019-09-06 21:50:00,100001510,spices,1,checkout


In [25]:
round(pd.crosstab(df_evening['location'], df_evening['shift'], normalize=0), 2)

shift,checkout,dairy,drinks,fruit,spices
location,,,,,
checkin,0.00,0.28,0.19,0.35,0.18
dairy,0.36,0.00,0.26,0.18,0.20
drinks,0.54,0.04,0.00,0.21,0.20
fruit,0.49,0.24,0.14,0.00,0.13
spices,0.27,0.31,0.28,0.15,0.00


In [26]:
pd.crosstab(df_evening['location'], df_evening['timestamp'].dt.hour)

timestamp,18,19,20,21
location,,,,
checkin,648,749,478,278
checkout,630,755,492,333
dairy,425,466,298,151
drinks,361,441,307,180
fruit,453,507,312,173
spices,346,389,246,122


### T matrix for whole week

In [27]:
df.sort_values(by=['ID', 'timestamp'])

,ID,checkout,checkout_at,customer_no,day,enter_at,entry,location,timestamp,week_code,time_spent,time_spent_m,sequence,tod
0,000010001,False,2019-09-04 07:02:00,1,wednesday,2019-09-04 07:00:00,True,checkin,2019-09-04 07:00:00,00001,0 days 00:02:00,2,0,morning
1,000010001,False,2019-09-04 07:02:00,1,wednesday,2019-09-04 07:00:00,True,fruit,2019-09-04 07:00:00,00001,0 days 00:02:00,2,1,morning
2,000010001,True,2019-09-04 07:02:00,1,wednesday,2019-09-04 07:00:00,False,checkout,2019-09-04 07:02:00,00001,0 days 00:02:00,2,2,morning
3,000010002,False,2019-09-04 07:06:00,2,wednesday,2019-09-04 07:00:00,True,checkin,2019-09-04 07:00:00,00001,0 days 00:06:00,6,0,morning
4,000010002,False,2019-09-04 07:06:00,2,wednesday,2019-09-04 07:00:00,True,dairy,2019-09-04 07:00:00,00001,0 days 00:06:00,6,1,morning
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32319,100001509,True,2019-09-06 21:50:00,1509,friday,2019-09-06 21:50:00,True,drinks,2019-09-06 21:50:00,10000,0 days 00:00:00,0,1,evening
32348,100001509,True,2019-09-06 21:50:00,1509,friday,2019-09-06 21:50:00,True,checkout,2019-09-06 21:51:00,10000,0 days 00:00:00,0,2,evening
32320,100001510,False,2019-09-06 21:50:00,1510,friday,2019-09-06 21:50:00,True,checkin,2019-09-06 21:50:00,10000,0 days 00:00:00,0,0,evening
32321,100001510,True,2019-09-06 21:50:00,1510,friday,2019-09-06 21:50:00,True,spices,2019-09-06 21:50:00,10000,0 days 00:00:00,0,1,evening


In [28]:
df['shift'] = df.groupby(['ID'])['location'].shift(-1)

In [29]:
df.sort_values(by=['ID', 'timestamp'])

,ID,checkout,checkout_at,customer_no,day,enter_at,entry,location,timestamp,week_code,time_spent,time_spent_m,sequence,tod,shift
0,000010001,False,2019-09-04 07:02:00,1,wednesday,2019-09-04 07:00:00,True,checkin,2019-09-04 07:00:00,00001,0 days 00:02:00,2,0,morning,fruit
1,000010001,False,2019-09-04 07:02:00,1,wednesday,2019-09-04 07:00:00,True,fruit,2019-09-04 07:00:00,00001,0 days 00:02:00,2,1,morning,checkout
2,000010001,True,2019-09-04 07:02:00,1,wednesday,2019-09-04 07:00:00,False,checkout,2019-09-04 07:02:00,00001,0 days 00:02:00,2,2,morning,NaN
3,000010002,False,2019-09-04 07:06:00,2,wednesday,2019-09-04 07:00:00,True,checkin,2019-09-04 07:00:00,00001,0 days 00:06:00,6,0,morning,dairy
4,000010002,False,2019-09-04 07:06:00,2,wednesday,2019-09-04 07:00:00,True,dairy,2019-09-04 07:00:00,00001,0 days 00:06:00,6,1,morning,spices
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32319,100001509,True,2019-09-06 21:50:00,1509,friday,2019-09-06 21:50:00,True,drinks,2019-09-06 21:50:00,10000,0 days 00:00:00,0,1,evening,checkout
32348,100001509,True,2019-09-06 21:50:00,1509,friday,2019-09-06 21:50:00,True,checkout,2019-09-06 21:51:00,10000,0 days 00:00:00,0,2,evening,NaN
32320,100001510,False,2019-09-06 21:50:00,1510,friday,2019-09-06 21:50:00,True,checkin,2019-09-06 21:50:00,10000,0 days 00:00:00,0,0,evening,spices
32321,100001510,True,2019-09-06 21:50:00,1510,friday,2019-09-06 21:50:00,True,spices,2019-09-06 21:50:00,10000,0 days 00:00:00,0,1,evening,checkout


In [47]:
pd.crosstab(df['location'], df['shift'], normalize=0)

shift,checkout,dairy,drinks,fruit,spices
location,,,,,
checkin,0.000000,0.287576,0.153526,0.377435,0.181464
dairy,0.393033,0.000000,0.222483,0.189357,0.195127
drinks,0.537260,0.027145,0.000000,0.218950,0.216645
fruit,0.500195,0.237993,0.136080,0.000000,0.125732
spices,0.251998,0.323122,0.272776,0.152104,0.000000


In [48]:
T_matrix = pd.crosstab(df['location'], df['shift'], normalize=0)

In [49]:
T_matrix.to_csv('t_matrix.csv')

In [31]:
pd.crosstab(df['location'], df['timestamp'].dt.hour)

timestamp,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
location,,,,,,,,,,,,,,,
checkin,492,707,453,395,336,396,531,485,399,534,564,648,749,478,278
checkout,454,677,455,427,342,384,514,496,412,518,556,630,755,492,333
dairy,275,455,291,294,210,225,340,319,233,337,360,425,466,298,151
drinks,278,348,231,198,150,188,238,229,190,272,294,361,441,307,180
fruit,299,468,336,270,244,247,389,345,316,353,410,453,507,312,173
spices,238,342,207,242,165,208,273,261,184,262,269,346,389,246,122


## Time spent in each section

In [32]:
df['time_shift'] = df.groupby(['ID'])['timestamp'].shift(-1)

In [33]:
df.sort_values(by=['ID', 'timestamp'])

,ID,checkout,checkout_at,customer_no,day,enter_at,entry,location,timestamp,week_code,time_spent,time_spent_m,sequence,tod,shift,time_shift
0,000010001,False,2019-09-04 07:02:00,1,wednesday,2019-09-04 07:00:00,True,checkin,2019-09-04 07:00:00,00001,0 days 00:02:00,2,0,morning,fruit,2019-09-04 07:00:00
1,000010001,False,2019-09-04 07:02:00,1,wednesday,2019-09-04 07:00:00,True,fruit,2019-09-04 07:00:00,00001,0 days 00:02:00,2,1,morning,checkout,2019-09-04 07:02:00
2,000010001,True,2019-09-04 07:02:00,1,wednesday,2019-09-04 07:00:00,False,checkout,2019-09-04 07:02:00,00001,0 days 00:02:00,2,2,morning,NaN,NaT
3,000010002,False,2019-09-04 07:06:00,2,wednesday,2019-09-04 07:00:00,True,checkin,2019-09-04 07:00:00,00001,0 days 00:06:00,6,0,morning,dairy,2019-09-04 07:00:00
4,000010002,False,2019-09-04 07:06:00,2,wednesday,2019-09-04 07:00:00,True,dairy,2019-09-04 07:00:00,00001,0 days 00:06:00,6,1,morning,spices,2019-09-04 07:05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32319,100001509,True,2019-09-06 21:50:00,1509,friday,2019-09-06 21:50:00,True,drinks,2019-09-06 21:50:00,10000,0 days 00:00:00,0,1,evening,checkout,2019-09-06 21:51:00
32348,100001509,True,2019-09-06 21:50:00,1509,friday,2019-09-06 21:50:00,True,checkout,2019-09-06 21:51:00,10000,0 days 00:00:00,0,2,evening,NaN,NaT
32320,100001510,False,2019-09-06 21:50:00,1510,friday,2019-09-06 21:50:00,True,checkin,2019-09-06 21:50:00,10000,0 days 00:00:00,0,0,evening,spices,2019-09-06 21:50:00
32321,100001510,True,2019-09-06 21:50:00,1510,friday,2019-09-06 21:50:00,True,spices,2019-09-06 21:50:00,10000,0 days 00:00:00,0,1,evening,checkout,2019-09-06 21:51:00


In [34]:
df['time_spent'] = (pd.to_datetime(df['time_shift']) - pd.to_datetime(df['timestamp']))

In [35]:
df['time_spent'].fillna(method='pad', inplace=True)

In [36]:
df

,ID,checkout,checkout_at,customer_no,day,enter_at,entry,location,timestamp,week_code,time_spent,time_spent_m,sequence,tod,shift,time_shift
0,000010001,False,2019-09-04 07:02:00,1,wednesday,2019-09-04 07:00:00,True,checkin,2019-09-04 07:00:00,00001,0 days 00:00:00,2,0,morning,fruit,2019-09-04 07:00:00
1,000010001,False,2019-09-04 07:02:00,1,wednesday,2019-09-04 07:00:00,True,fruit,2019-09-04 07:00:00,00001,0 days 00:02:00,2,1,morning,checkout,2019-09-04 07:02:00
2,000010001,True,2019-09-04 07:02:00,1,wednesday,2019-09-04 07:00:00,False,checkout,2019-09-04 07:02:00,00001,0 days 00:02:00,2,2,morning,NaN,NaT
3,000010002,False,2019-09-04 07:06:00,2,wednesday,2019-09-04 07:00:00,True,checkin,2019-09-04 07:00:00,00001,0 days 00:00:00,6,0,morning,dairy,2019-09-04 07:00:00
4,000010002,False,2019-09-04 07:06:00,2,wednesday,2019-09-04 07:00:00,True,dairy,2019-09-04 07:00:00,00001,0 days 00:05:00,6,1,morning,spices,2019-09-04 07:05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32319,100001509,True,2019-09-06 21:50:00,1509,friday,2019-09-06 21:50:00,True,drinks,2019-09-06 21:50:00,10000,0 days 00:01:00,0,1,evening,checkout,2019-09-06 21:51:00
32348,100001509,True,2019-09-06 21:50:00,1509,friday,2019-09-06 21:50:00,True,checkout,2019-09-06 21:51:00,10000,0 days 00:01:00,0,2,evening,NaN,NaT
32320,100001510,False,2019-09-06 21:50:00,1510,friday,2019-09-06 21:50:00,True,checkin,2019-09-06 21:50:00,10000,0 days 00:00:00,0,0,evening,spices,2019-09-06 21:50:00
32321,100001510,True,2019-09-06 21:50:00,1510,friday,2019-09-06 21:50:00,True,spices,2019-09-06 21:50:00,10000,0 days 00:01:00,0,1,evening,checkout,2019-09-06 21:51:00


In [37]:
df['time_spent'] = df['time_spent']/pd.Timedelta('60s')

In [38]:
df.drop('time_spent_m', axis=1, inplace=True)

In [44]:
df.to_hdf('df_input_class.h5', key='df', mode='w', format='table')

## T-matrix for time spent

In [120]:
df.groupby(['location'])['time_spent'].mean()

location
checkin     0.000000
checkout    2.706380
dairy       3.798675
drinks      2.490653
fruit       2.481062
spices      1.672349
Name: time_spent, dtype: float64

In [128]:
pd.crosstab(df['location'], df['time_spent'], normalize=0)

time_spent,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,19.0,20.0,21.0,23.0,24.0,25.0,26.0,27.0,28.0,30.0
location,,,,,,,,,,,,,,,,,,,,,
checkin,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
checkout,0.0,0.392478,0.231833,0.141169,0.078576,0.049966,0.035057,0.024177,0.014506,0.011283,...,0.000403,0.000134,0.000000,0.000269,0.000134,0.000000,0.000000,0.000000,0.000000,0.000134
dairy,0.0,0.296217,0.185937,0.134217,0.090831,0.070314,0.050652,0.045095,0.030135,0.027143,...,0.001496,0.000427,0.000427,0.000641,0.000427,0.000214,0.000641,0.000214,0.000214,0.000214
drinks,0.0,0.388220,0.255570,0.147759,0.082458,0.050192,0.030218,0.016901,0.011268,0.008195,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
fruit,0.0,0.410972,0.229988,0.147403,0.083756,0.047052,0.034947,0.020109,0.011519,0.005662,...,0.000390,0.000195,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
spices,0.0,0.600959,0.238146,0.096430,0.039158,0.012520,0.007991,0.002930,0.000533,0.000266,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [137]:
time_spent = pd.crosstab(df['location'], df['time_spent'], normalize=0)

In [138]:
time_spent.reset_index()

time_spent,location,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,...,19.0,20.0,21.0,23.0,24.0,25.0,26.0,27.0,28.0,30.0
0,checkin,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,checkout,0.0,0.392478,0.231833,0.141169,0.078576,0.049966,0.035057,0.024177,0.014506,...,0.000403,0.000134,0.000000,0.000269,0.000134,0.000000,0.000000,0.000000,0.000000,0.000134
2,dairy,0.0,0.296217,0.185937,0.134217,0.090831,0.070314,0.050652,0.045095,0.030135,...,0.001496,0.000427,0.000427,0.000641,0.000427,0.000214,0.000641,0.000214,0.000214,0.000214
3,drinks,0.0,0.388220,0.255570,0.147759,0.082458,0.050192,0.030218,0.016901,0.011268,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,fruit,0.0,0.410972,0.229988,0.147403,0.083756,0.047052,0.034947,0.020109,0.011519,...,0.000390,0.000195,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,spices,0.0,0.600959,0.238146,0.096430,0.039158,0.012520,0.007991,0.002930,0.000533,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [142]:
time_spent.drop(time_spent[0], axis=1)

time_spent,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,...,19.0,20.0,21.0,23.0,24.0,25.0,26.0,27.0,28.0,30.0
location,,,,,,,,,,,,,,,,,,,,,
checkin,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
checkout,0.231833,0.141169,0.078576,0.049966,0.035057,0.024177,0.014506,0.011283,0.006044,0.004030,...,0.000403,0.000134,0.000000,0.000269,0.000134,0.000000,0.000000,0.000000,0.000000,0.000134
dairy,0.185937,0.134217,0.090831,0.070314,0.050652,0.045095,0.030135,0.027143,0.016029,0.013037,...,0.001496,0.000427,0.000427,0.000641,0.000427,0.000214,0.000641,0.000214,0.000214,0.000214
drinks,0.255570,0.147759,0.082458,0.050192,0.030218,0.016901,0.011268,0.008195,0.004097,0.002305,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
fruit,0.229988,0.147403,0.083756,0.047052,0.034947,0.020109,0.011519,0.005662,0.003124,0.001757,...,0.000390,0.000195,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
spices,0.238146,0.096430,0.039158,0.012520,0.007991,0.002930,0.000533,0.000266,0.000533,0.000266,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [143]:
time_spent.index

Index(['checkin', 'checkout', 'dairy', 'drinks', 'fruit', 'spices'], dtype='object', name='location')

In [144]:
time_spent.to_csv('time_spent.csv')